# ランダムサーチ（Random Search）
探索空間からランダムにサンプルして試す方法  
ランダムに探索するため、グリッドサーチよりも計算コストを抑えられる  
再現性に注意が必要

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# シード固定
torch.manual_seed(42)
np.random.seed(42)

# サンプルデータ生成：y = 2*x + 1 にノイズを加えた回帰データ
X = np.linspace(-1, 1, 100).reshape(-1, 1)
y = 2 * X + 1 + np.random.normal(0, 0.2, X.shape)

# NumPy → PyTorch tensor
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

# シンプルなMLPモデル（1層の隠れ層）
class SimpleMLP(nn.Module):
    def __init__(self, hidden_size):
        super(SimpleMLP, self).__init__()
        self.fc1 = nn.Linear(1, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, 1)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# モデル訓練関数：指定したエポック数だけ訓練し、最終MSEを返す
def train_model(hidden_size, lr, epochs=100):
    model = SimpleMLP(hidden_size)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        output = model(X_tensor)
        loss = criterion(output, y_tensor)
        loss.backward()
        optimizer.step()
    return loss.item()


In [4]:
import random

n_trials = 10
best_loss = float('inf')
best_params = None

for i in range(n_trials):
    hidden = random.choice(range(4, 33))  # 4～32の整数
    lr = 10 ** random.uniform(-4, -1)      # 1e-4～1e-1 の範囲
    loss = train_model(hidden, lr, epochs=100)
    print(f"Trial {i+1}: Hidden: {hidden}, LR: {lr:.5f} -> Loss: {loss:.4f}")
    if loss < best_loss:
        best_loss = loss
        best_params = (hidden, lr)

print("Best parameters (Random Search):", best_params, "with Loss:", best_loss)


Trial 1: Hidden: 8, LR: 0.00073 -> Loss: 1.5896
Trial 2: Hidden: 19, LR: 0.00282 -> Loss: 0.2414
Trial 3: Hidden: 13, LR: 0.00015 -> Loss: 2.5565
Trial 4: Hidden: 23, LR: 0.00016 -> Loss: 2.2540
Trial 5: Hidden: 18, LR: 0.00590 -> Loss: 0.0398
Trial 6: Hidden: 17, LR: 0.00031 -> Loss: 2.2928
Trial 7: Hidden: 24, LR: 0.00072 -> Loss: 1.4901
Trial 8: Hidden: 5, LR: 0.00065 -> Loss: 1.7357
Trial 9: Hidden: 22, LR: 0.02466 -> Loss: 0.0304
Trial 10: Hidden: 12, LR: 0.08978 -> Loss: 0.0309
Best parameters (Random Search): (22, 0.024658703064721996) with Loss: 0.030424686148762703
